In [53]:
#Глубокое обучение - итоговый проект

In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
test = test.drop('ID', axis=1)

In [56]:
test = pd.read_csv("data_deap/test.csv")
test['chest'] = test['chest'].apply(lambda x: 0 if x < 0 else x)

#окрглим признак "chest" до 2 знаков после ,
test['chest'] = test['chest'].round(2)
#округлим признак "resting_blood_pressure"
test['resting_blood_pressure'] = test['resting_blood_pressure'].round()
#округлим признак "serum_cholestoral" 
test['serum_cholestoral'] = test['serum_cholestoral'].round(1)
#округлим признак "maximum_heart_rate_achieved"
test['maximum_heart_rate_achieved'] = test['maximum_heart_rate_achieved'].round()
#округлим признак "oldpeak"
test['oldpeak'] = test['oldpeak'].round(2)
test['age'] = test['age'].round()

In [57]:
#создаём dummies
df = test['thal']
df = pd.get_dummies(df, columns='thal', drop_first=True)
test['fix_def'] = df[6]
test['rev_def'] = df[7]
test = test.drop('thal', axis=1)

df = test['slope']
df = pd.get_dummies(df, columns='slope', drop_first=True)
test['slope=2'] = df[2]
test['slope=3'] = df[3]
test = test.drop('slope', axis=1)

df = test['resting_electrocardiographic_results'] #делаем тоже самое для resting_electrocardiographic_results
df = pd.get_dummies(df, columns='resting_electrocardiographic_results', drop_first=True)
test['rest_elcardio=1'] = df[1]
test['rest_elcardio=2'] = df[2]
test = test.drop('resting_electrocardiographic_results', axis=1)



In [58]:
#Создадим функцию для первода dummies в формат 1/0, а потом в категории
def dum_to_cat(data, columns):
    for i in columns:
        data[i] = data[i].astype(int)
        data[i] = data[i].astype('category')
    return data

In [59]:
#Перевоодим dummies в формат 1/0 а затем в тип category 
columns = ['fix_def', 'rev_def', 'slope=2', 'slope=3', 'rest_elcardio=1', 'rest_elcardio=2']
new_data = dum_to_cat(test, columns)

In [60]:
#Создаём категории
test['fasting_blood_sugar'] = test['fasting_blood_sugar'].astype('category')
test['exercise_induced_angina'] = test['exercise_induced_angina'].astype('category')
test['number_of_major_vessels'] = test['number_of_major_vessels'].astype('category')
test['sex'] = test['sex'].astype('category')


In [61]:
#Удалим мультиколенеарные признаки и малозначимые признаки
test = test.drop('exercise_induced_angina', axis=1)
test = test.drop(columns=['fasting_blood_sugar', 'maximum_heart_rate_achieved'], axis=1)

Подготовим данные к моделированию

In [79]:
X_test = test

In [81]:
#переведём данные в формат массива
X_test = np.asarray(X_test)

PyTorch

In [64]:
!pip install torch torchvision torchaudio


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [73]:
#Предсказание y_test сделаем на модели PyTorch
model = BinaryClassifier(input_size=(600000, 13)[1])

In [84]:
#Переведём данные в нужный формат
X_test = torch.tensor(X_test, dtype=torch.float32)

In [80]:
X_test.shape

(400000, 13)

In [69]:
model.load_state_dict(torch.load('PyTorch_model', weights_only=True))

<All keys matched successfully>

In [70]:
#Выгрузим модель

model.load_state_dict(torch.load('PyTorch_model'))

C:\Users\lenovo\AppData\Local\Temp\ipykernel_1544\3630251337.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('PyTorch_model'))


<All keys matched successfully>

Предсказание

In [85]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predicted_classes = (predictions > 0.5).float()

In [ ]:
#Предсказание находится в переменной predicted_classes